In [ ]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import time


def crawl_seoul_housing():
    """
    - 대상: 신혼부부/청년/대학생
    - 나머지 조건: 전체(미선택)

    - 정책명: div.srl-left > p.art-tit
    - 요약:   div.srl-left > p.art-descrip
    - 링크:   div.srl-right > a[href]
    """

    base_url = "https://housing.seoul.go.kr/site/main/polcyManageInfo/customized/list"

    # 대상자 코드
    target_groups = {
        "신혼부부": "T_3",
        "청년": "T_4",
        "대학생": "T_11",
    }

    headers = {
        "User-Agent": (
            "Mozilla/5.0 (Windows NT 10.0; Win64; x64) "
            "AppleWebKit/537.36 (KHTML, like Gecko) "
            "Chrome/120.0.0.0 Safari/537.36"
        ),
        "Referer": base_url,
    }

    all_data = []

    for group_name, group_code in target_groups.items():
        print(f"\n========================================")
        print(f"[{group_name}] (코드: {group_code}) 수집 시작")
        print(f"========================================")

        cp = 1
        last_first_link = None 

        while True:
            params = {
                "trgterSearch": group_code,
                "cp": cp,
                "useYnSearch": "Y",
            }

            resp = requests.get(base_url, params=params, headers=headers, timeout=20)
            resp.raise_for_status()
            soup = BeautifulSoup(resp.text, "html.parser")

            list_ul = soup.select_one("ul.sr-list")
            if not list_ul:
                print("  └ [종료] ul.sr-list 없음 (마지막 페이지 가능)")
                break

            items = list_ul.select("li.clearfix")
            if not items:
                print("  └ [종료] items 없음 (마지막 페이지 가능)")
                break

            # 중복 페이지 검사: 첫 번째 항목의 링크가 이전과 같으면 서버가 같은 페이지를 반환한 것
            first_a = items[0].select_one("div.srl-right a[href]")
            first_link = first_a.get("href").strip() if first_a and first_a.get("href") else None

            if first_link == last_first_link:
                print(f"  └ {cp}페이지: 이전 페이지와 동일 → 수집 종료")
                break
            last_first_link = first_link

            print(f"  └ {cp}페이지 수집 중...")

            page_count = 0

            for item in items:
                left = item.select_one("div.srl-left")
                right = item.select_one("div.srl-right")

                # -----------------------------
                # 1) 정책명(title): p.art-tit
                # -----------------------------
                title = ""
                if left:
                    t = left.select_one("p.art-tit")
                    if t:
                        title = t.get_text(" ", strip=True)

                # -----------------------------
                # 2) 요약(summary): p.art-descrip
                # -----------------------------
                summary = ""
                if left:
                    d = left.select_one("p.art-descrip")
                    if d:
                        summary = d.get_text(" ", strip=True)

                # -----------------------------
                # 3) 링크(link): div.srl-right > a[href]
                #    - a 텍스트는 "새 창 열림"이라 제목으로 쓰면 안 됨
                # -----------------------------
                link = ""
                if right:
                    a = right.select_one("a[href]")
                    if a:
                        link = (a.get("href") or "").strip()

                # 링크 절대경로 보정
                if link and not link.startswith("http"):
                    link = f"https://housing.seoul.go.kr{link}"

                # -----------------------------
                # 빈 행 제거 로직
                # - 마지막 행이 빈 데이터로 들어오는 문제 방지
                # - title도 없고 link도 없으면 의미 없는 row이므로 스킵
                # -----------------------------
                if not title and not link:
                    continue

                # summary가 title과 같아지는 경우(혹시라도) 방지
                if summary == title:
                    summary = ""

                all_data.append(
                    {
                        "대상": group_name,
                        "정책명": title,
                        "요약": summary,
                        "링크": link,
                    }
                )
                page_count += 1

            print(f"    └ {cp}페이지에서 {page_count}건 수집")
            cp += 1
            time.sleep(0.5)

    return pd.DataFrame(all_data)


if __name__ == "__main__":
    df = crawl_seoul_housing()
    print(f"\n총 {len(df)}개의 정책을 수집했습니다.")

    filename = "link_parsed.csv"
    df.to_csv(filename, index=False, encoding="utf-8-sig")
    print(f"파일 저장 완료: {filename}")


[신혼부부] (코드: T_3) 수집 시작
  └ 1페이지 수집 중...
    └ 1페이지에서 10건 수집
  └ 2페이지 수집 중...
    └ 2페이지에서 10건 수집
  └ 3페이지 수집 중...
    └ 3페이지에서 9건 수집
  └ 4페이지 수집 중...
    └ 4페이지에서 0건 수집
  └ 5페이지: 이전 페이지와 동일 → 수집 종료

[청년] (코드: T_4) 수집 시작
  └ 1페이지 수집 중...
    └ 1페이지에서 10건 수집
  └ 2페이지 수집 중...
    └ 2페이지에서 10건 수집
  └ 3페이지 수집 중...
    └ 3페이지에서 10건 수집
  └ 4페이지 수집 중...
    └ 4페이지에서 1건 수집
  └ 5페이지 수집 중...
    └ 5페이지에서 0건 수집
  └ 6페이지: 이전 페이지와 동일 → 수집 종료

[대학생] (코드: T_11) 수집 시작
  └ 1페이지 수집 중...
    └ 1페이지에서 10건 수집
  └ 2페이지 수집 중...
    └ 2페이지에서 10건 수집
  └ 3페이지 수집 중...
    └ 3페이지에서 9건 수집
  └ 4페이지 수집 중...
    └ 4페이지에서 0건 수집
  └ 5페이지: 이전 페이지와 동일 → 수집 종료

총 89개의 정책을 수집했습니다.
파일 저장 완료: seoul_housing_policies_final.csv
